In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
rrank = lambda df,k: df[np.isclose(df['rrank'], 1/k)]
equal = lambda df,k: df[np.isclose(df['rrank'], 1/k)]
gequal = lambda df,k: df[np.isclose(df['rrank'], 1/k) | (df['rrank'].ge(1/k))]

In [3]:
all = pd.read_csv('/data/khodadaa/lucene-index/rranks.csv', header=None)
sub = pd.read_csv('/data/khodadaa/lucene-index/rr/05092047_sub2_rranks.csv', header=None)
all.rename(columns={0:'query', 1: 'prec@10', 2: 'prec@20', 3: 'rrank', 4: 'avg(prec)', 5: 'recall@200', 6: 'recall@1000', 7:'ndcg10'}, inplace=True)
sub.rename(columns={0:'query', 1: 'prec@10', 2: 'prec@20', 3: 'rrank', 4: 'avg(prec)', 5: 'recall@200', 6: 'recall@1000', 7:'ndcg10'}, inplace=True)

In [4]:
a1 = equal(all, 1)
s1 = equal(sub, 1)
a2 = gequal(all, 2)
s2 = gequal(sub, 2)
a3 = gequal(all, 3)
s3 = gequal(sub, 3)

In [5]:
query_assignment = pd.read_csv('/data/khodadaa/lucene-index/query-assignments/05092047_query_assignment.csv', header=None)
query_assignment.rename(columns={0: 'query', 1:'assignment'}, inplace=True)
query_assignment['assignment'] = query_assignment['assignment'].map(lambda x: x.strip())
query_assignment['query'] = query_assignment['query'].map(lambda x: x.strip())
query_assignment['rrank'] = -1.0
for index, row in query_assignment.iterrows():    
    if row['assignment'] == 'all':        
        rrv = all.loc[all['query']==row['query'], 'rrank']
    elif row['assignment'] == 'sub':        
        rrv = sub.loc[sub['query']==row['query'], 'rrank']
    if rrv.size == 0:
        continue
    query_assignment.loc[index, 'rrank'] = rrv.values[0]

In [6]:
tran1_sub = sub[sub['query'].isin(a1['query'])]
tran2_sub = sub[sub['query'].isin(a2['query'])]
tran3_sub = sub[sub['query'].isin(a3['query'])]
#query_assignment[query_assignment['rrank'] < 0.0]
tran1_alg = query_assignment[query_assignment['query'].isin(a1['query'])]
tran2_alg = query_assignment[query_assignment['query'].isin(a2['query'])]
tran3_alg = query_assignment[query_assignment['query'].isin(a3['query'])]

In [12]:
print(a1.shape[0], a2.shape[0], a3.shape[0])
print(s1.shape[0], s2.shape[0], s3.shape[0])
print(a1['rrank'].mean(), a2['rrank'].mean(), a3['rrank'].mean())
print(tran1_sub['rrank'].mean(), tran2_sub['rrank'].mean(), tran3_sub['rrank'].mean())
print(tran1_alg['rrank'].mean(), tran2_alg['rrank'].mean(), tran3_alg['rrank'].mean())

972 1657 2134
4147 4677 4924
1.0 0.7933011466505733 0.6904873477038426
0.8401577503429355 0.8500301750150875 0.8577527170785445
0.9008573388203017 0.8618990142828405 0.8469748351666421


In [20]:
from scipy.stats import ttest_rel
def t_test(data1, data2, alpha=0.05):
    # compare samples
    stat, p = ttest_rel(data1, data2)
    print('Statistics=%.3f, p=%.5f' % (stat, p))
    # interpret    
    if p > alpha:
        print('Same distributions (fail to reject H0)')
    else:
        print('Different distributions (reject H0)')
t_test(a1['rrank'], tran1_sub['rrank'])
t_test(a1['rrank'], tran1_alg['rrank'])
t_test(a2['rrank'], tran2_sub['rrank'])
t_test(a2['rrank'], tran2_alg['rrank'])


Statistics=13.831, p=0.00000
Different distributions (reject H0)
Statistics=10.582, p=0.00000
Different distributions (reject H0)
Statistics=-5.347, p=0.00000
Different distributions (reject H0)
Statistics=-7.023, p=0.00000
Different distributions (reject H0)


# Tricks

In [ ]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
ts = ts.cumsum()
ts.plot()
plt.figure()

In [ ]:
%%bash
find /data/khodadaa/analysis/ -name "*.csv"

In [ ]:
%%latex
\begin{equation}
maximize \quad KLD(C,D) + \lambda \Sigma pop(d, C)\\
s.t. \quad \forall d \in C pop(d,C) \geq \theta_p
\end{equation}